In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib
import re
from datetime import datetime
from wordcloud import WordCloud 
import jieba
import jieba.analyse
from datetime import date
import re
import string
from wordcloud import WordCloud 
import json

### 構面字典

In [7]:
# dict_ = pd.read_csv('../構面字典-new.csv')
dict_ = pd.read_csv('./aspect.csv')
df_dict = dict_[['name','alias']]
d_list = df_dict.to_dict(orient='records')
structure_dict = {} # 構面字典 => name:alias
for i in d_list:
    structure_dict[i['name']] = i['alias']
structure_dict.keys()#key:成就感,...

dict_keys(['成就感', '學習成長', '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模', '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全', '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種', '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標', '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員', '正向', '負向'])

In [8]:
structure_origin = dict_[['class','name']]
structures_list = structure_origin['class'].unique().tolist() # 有哪些構面字陣列

structure_origin_list = pd.DataFrame(structure_origin.groupby('class')['name'].apply(list)).reset_index().to_dict(orient='records')
transformed_data = [{item['class']: item['name']} for item in structure_origin_list] # 取得構面字典，型態為list of dict

In [11]:
new_dict = {}
for i in transformed_data:
    new_dict[list(i.keys())[0]] = list(i.values())[0]
new_dict

{'人際價值': ['工作氛圍', '同事互動', '主管風格'],
 '公司文化與價值': ['多元性', '企業社會責任', '企業永續目標'],
 '情緒': ['正向', '負向'],
 '成長價值': ['成就感', '學習成長', '創新'],
 '招募人員': ['招募人員'],
 '招募資訊': ['面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力'],
 '產品與服務': ['引擎',
  '馬力',
  '避震',
  '外觀',
  '操控',
  '安全',
  '堅固',
  '配備',
  '價錢',
  '科技',
  '品質',
  '折舊',
  '品牌',
  '空間',
  '保養',
  '續航',
  '尺寸',
  '車種',
  '驅動',
  '變速箱'],
 '科系專業': ['電資', '機械', '製造', '車輛工程', '品管'],
 '組織安全與安定': ['工作地點', '公司規模', '工作環境', '產業前景'],
 '經濟報酬': ['薪資', '福利', '管理制度'],
 '變化性': ['輪調', '外派', '出差']}

In [12]:
tf = open("aspect.json", "w")
json.dump(new_dict,tf)

In [14]:
tf = open("aspect.json", "r")
new_dict = json.load(tf)
print(new_dict)

{'人際價值': ['工作氛圍', '同事互動', '主管風格'], '公司文化與價值': ['多元性', '企業社會責任', '企業永續目標'], '情緒': ['正向', '負向'], '成長價值': ['成就感', '學習成長', '創新'], '招募人員': ['招募人員'], '招募資訊': ['面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力'], '產品與服務': ['引擎', '馬力', '避震', '外觀', '操控', '安全', '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種', '驅動', '變速箱'], '科系專業': ['電資', '機械', '製造', '車輛工程', '品管'], '組織安全與安定': ['工作地點', '公司規模', '工作環境', '產業前景'], '經濟報酬': ['薪資', '福利', '管理制度'], '變化性': ['輪調', '外派', '出差']}


In [22]:
# 以pattern_text 例如是成就感，那就會去找text中有出現成就感的alias的次數
def get_dict_num(pattern_text, text):
    pattern = r"(" + structure_dict[pattern_text] + ")"
    num = len(re.findall(pattern, text))
    return num

In [23]:
# 取得資料
df_ = pd.read_csv('./interview_clean.csv')
df_.drop(['Unnamed: 0','rank','difficult'], axis=1,inplace=True)
df_.rename(columns={'whole_art':'content'}, inplace=True)
df_ = df_[~df_['process'].isna()].reset_index(drop='index') # (7219 , 8)
df_.head()
df_
# df_ = df_[~df_['month_salary'].isna()].reset_index(drop='index')
# df_.drop(['month_salary','year_salary','year-end_salary','bonus','dividends','other','average_day','seniority','total_seniority','mood'], axis = 1, inplace=True)

,aID,company_name,vacancies,post_time,interview_time,status,process,question,suggestion,content,word
0,0,裕隆集團_納智捷汽車股份有限公司,企劃管理師,2022.08.24,2022.07,未錄取,針對專業提出疑問，並...,"[{'Q': '這個品牌業界傳的都不太好，為什麼願意來納智捷面試', 'A': '因為我本來...",NaN,針對專業提出疑問，並。這個品牌業界傳的都不太好，為什麼願意來納智捷面試？因為我本來就愛納智捷。,"['專業', '提出', '疑問', '品牌', '業界', '願意', '納智捷', '面..."
1,1,裕隆集團_納智捷汽車股份有限公司,管理師,2022.02.21,2019.02,無聲卡,整體面試過程約2小時...,"[{'Q': '對於納智捷的了解', 'A': '依照職位跟公司官網做...'}]",NaN,整體面試過程約2小時。對於納智捷的了解？依照職位跟公司官網做。,"['體面', '過程', '小時', '納智捷', '職位', '公司', '官網']"
2,2,裕隆集團_納智捷汽車股份有限公司,行銷專員,2022.01.13,2017.09,錄取,第一次面試先簡單自我...,"[{'Q': '會不會開車', 'A': '有駕照，但當時還不熟...'}]",NaN,第一次面試先簡單自我。會不會開車？有駕照，但當時還不熟。,"['面試', '簡單', '自我', '會不會', '開車', '駕照', '熟']"
3,3,裕隆集團_納智捷汽車股份有限公司,人力資源管理師,2021.09.12,2021.09,未錄取,第一關，人資面試\n...,"[{'Q': '結算薪資怎麼做', 'A': '照実回答'}]",NaN,第一關，人資面試。。結算薪資怎麼做？照実回答。,"['人資', '面試', '結算', '薪資', '照実', '回答']"
4,4,裕隆集團_納智捷汽車股份有限公司,秘書,2021.04.21,2017.09,錄取,在台南東區的廠二樓面...,"[{'Q': '與人交談方面的問題', 'A': '我是個開朗的人，比較...'}]",NaN,在台南東區的廠二樓面。與人交談方面的問題？我是個開朗的人，比較。,"['台南', '東區', '二樓面', '交談', '方面', '問題', '開朗']"
...,...,...,...,...,...,...,...,...,...,...,...
7249,7636,匯豐汽車股份有限公司,產品企劃專員,2018.04.20,2017.12,錄取,簡報說明令人印象深刻,"[{'Q': '針對產品提出看法', 'A': '直接腦袋中的想法講出...'}]",NaN,簡報說明令人印象深刻。針對產品提出看法？直接腦袋中的想法講出。,"['簡報', '說明', '令', '印象深刻', '產品', '提出', '看法', '直..."
7250,7637,匯豐汽車股份有限公司,SUM服務專員,2018.03.23,2010.06,錄取,面試過程其實很好，而...,"[{'Q': '時間有點久不太記得，但是要看你遇到的廠長是好是壞!', 'A': '時間有點...",NaN,面試過程其實很好，而。時間有點久不太記得，但是要看你遇到的廠長是好是壞!？時間有點久不太記得，。,"['面試', '過程', '時間', '記得', '遇到', '廠長', '壞', '時間'..."
7251,7638,匯豐汽車股份有限公司,總公司-品牌部,2018.01.24,2016.06,錄取,帶到會議室，等待面試...,[{'Q': '對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行銷規劃...,NaN,帶到會議室，等待面試。對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行...,"['帶到', '會議室', '等待', '面試', '行銷', '品牌', '看法', 'E..."
7252,7639,匯豐汽車股份有限公司,技師,2017.12.24,2002.06,錄取,直接面試 問維修相關...,"[{'Q': '需配合值班可否接受 前份工作為何離職', 'A': '可以\n\n職涯規劃'}]",NaN,直接面試 問維修相關。需配合值班可否接受 前份工作為何離職？可以。職涯規劃。,"['直接面試問', '維修', '相關', '需', '配合', '值班', '可否', '..."


In [24]:
# 只用有解鎖的文章
# df_open = df_[df_['process'].apply(lambda x: isopen(x))].reset_index(drop='index')
df_open = df_[~df_['suggestion'].isna()]
df_open.shape

(536, 11)

In [25]:
# 字典型態的資料
# 資料轉為字典，比較方便去做計算
dict_ = df_open.to_dict(orient='records') 

In [26]:
# 取得根據每個構面下的類別名稱，計算該類別名稱出現多少次，來去計算dtm
for structure in list(structure_dict.keys()):
    for data in dict_:
        data[structure] = get_dict_num(structure, data['content'])

In [27]:
# DTM for 趨勢圖
# 把用不到的欄位踢除
kick_off_key = ['company_name','status','process','question','suggestion','frequency_overtime','Q_content']
data_list = [{k: v for k, v in d.items() if k not in kick_off_key} for d in dict_]
# 得到DTM DataFrame
dtm_df = pd.DataFrame(data_list)
dtm_df.shape

(536, 63)

In [28]:
# DTM for correlation & co-ocurence #把用不到的欄位踢除
kick_off_ = ['vacancies','post_time','interview_time','content','company_name','status','process','question','suggestion','frequency_overtime','Q_content', 'word', 'aID']
co_ = [{k: v for k, v in d.items() if k not in kick_off_} for d in dict_]
dtm_co = pd.DataFrame(co_)
dtm_co.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,0,0,0,0,0,2,0,0,2,1,...,1,0,0,0,0,0,6,3,2,218
1,0,0,0,3,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,2,241
2,0,0,0,0,0,0,1,0,2,0,...,0,0,0,0,0,0,0,2,3,192
3,0,2,0,0,0,5,0,1,2,0,...,2,0,0,0,0,0,4,3,12,647
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,2,1,189


In [29]:
dtm_df.head()

,aID,vacancies,post_time,interview_time,content,word,成就感,學習成長,創新,薪資,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,41,職員,2023.03.24,2023.03,首先會先做專業能力測驗，大約50分鐘，再來會做英文多益測驗40分鐘，做完後就進行視訊面試，1...,"['專業', '能力', '測驗', '分鐘', '英文', '多益', '測驗', '分鐘...",0,0,0,0,...,1,0,0,0,0,0,6,3,2,218
1,42,客服,2023.03.24,2023.01,覺得很浪費時間，我面試大夜班，到現場要先測試聽力打字，現場很吵，然後反應也沒用，也不知道在測...,"['浪費', '時間', '面試', '大夜班', '現場', '測試', '聽力', '打...",0,0,0,3,...,1,0,0,0,0,0,0,1,2,241
2,43,職員,2023.03.06,2018.01,主管是一個溫文儒雅的先生 真的感覺得出來他非常誠懇 然後問的問題沒有太多深入的題目 主要就是...,"['主管', '溫文儒雅', '先生', '感覺', '誠懇', '問題', '深入', '...",0,0,0,0,...,0,0,0,0,0,0,0,2,3,192
3,44,物料管理實習生,2023.03.02,2022.05,面試是以視訊面試進行，除了HR以外還有一個物料管理部門的主管（？）之類的長官，整體面試進行的...,"['面試', '視訊', '面試', 'HR', '物料', '管理', '部門', '主管...",0,2,0,0,...,2,0,0,0,0,0,4,3,12,647
4,45,職員,2023.02.16,2021.06,先遠端考英文及專業測驗，大約1個半小時，而後開始視訊面試。面談大約進行一小時，面試官有3-4...,"['遠', '端考', '英文', '專業', '測驗', '小時', '視訊', '面試'...",0,0,0,0,...,0,0,0,0,0,0,2,2,1,189


In [11]:
dtm_df = pd.read_csv('./salary/salary_sentence_dictword_dtm.csv')
dtm_df

,aID,company_name,vacancies,post_time,result,sentences,成就感,學習成長,創新,薪資,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,公司算是老公司所以做法非常老派但是非常穩定可以穩穩的待,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,1因為要解析車子的問題但是因為公司內部的流程做法讓解析的動作會變得非常的困難,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,2上層主管會使用各種活動減少成本及提高產能但是這個活動本身就會造成工程師很多的困擾卻還是一直...,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
3,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,如果想進裕隆的朋友是非常喜歡寫報告及報告的人那這一間你會非常適合同一份報告你可以寫三次分不同...,0,0,0,0,...,0,0,0,0,0,0,0,2,3,0
4,43,裕隆集團_裕隆汽車製造股份有限公司,工程師,2023.02.28,報告很多，實用價值低。會議很多，有開等於沒開，開完一堆追蹤。對主要專精車輛事業不如中華汽車。...,報告很多實用價值低,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,講真的每天保養維修車輛數又不多輪胎電瓶定位有報價客人也不一定要換因為外場便宜很多,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
363,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,況且車子已經賣不好了每個月廠業績越定越高根本很難達到,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
364,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,建議去別廠牌車子賣得比較好的保養廠做不然匯豐商用客戶已經慢慢流向豐田中華產品也沒在創新在沒幾...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
365,5931,匯豐汽車股份有限公司,助理,2022.11.11,行政作業居多，內業為主，但偶而要有外出的可能性，光是入貴公司，我的職位因為公司業務考量，直接...,行政作業居多內業為主但偶而要有外出的可能性光是入貴公司我的職位因為公司業務考量直接變更職務依...,0,0,0,1,...,0,1,0,0,0,0,0,0,1,2


In [12]:
def get_ym(time):
    l = time.split('.')
    return l[0]+"-"+l[1]

In [13]:
# # 新增年月日欄位
# date_format = '%Y.%m.%d'
# dtm_df['post_time'] = dtm_df['post_time'].apply(lambda x: datetime.strptime(x, date_format))
# dtm_df['p_year'] = dtm_df['post_time'].apply(lambda x: x.year)
# dtm_df['p_month'] = dtm_df['post_time'].apply(lambda x: x.month)
# dtm_df['p_day'] = dtm_df['post_time'].apply(lambda x: x.day)
dtm_df['p_year_month'] = dtm_df['post_time'].apply(lambda x: get_ym(x))
# dtm_df = dtm_df.reindex(columns=['vacancies', 'post_time', 'p_year_month', 'interview_time', 'content', 'word', '成就感', '學習成長',
#        '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模',
#        '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全',
#        '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種',
#        '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標',
#        '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員',
#        '正向', '負向'])
dtm_df

,aID,company_name,vacancies,post_time,result,sentences,成就感,學習成長,創新,薪資,...,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向,p_year_month
0,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,公司算是老公司所以做法非常老派但是非常穩定可以穩穩的待,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2023-03
1,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,1因為要解析車子的問題但是因為公司內部的流程做法讓解析的動作會變得非常的困難,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2023-03
2,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,2上層主管會使用各種活動減少成本及提高產能但是這個活動本身就會造成工程師很多的困擾卻還是一直...,0,0,0,0,...,0,0,0,0,0,0,1,0,2,2023-03
3,42,裕隆集團_裕隆汽車製造股份有限公司,生產技術工程師,2023.03.06,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待。1.因為要解析車子的問題但是因為...,如果想進裕隆的朋友是非常喜歡寫報告及報告的人那這一間你會非常適合同一份報告你可以寫三次分不同...,0,0,0,0,...,0,0,0,0,0,0,2,3,0,2023-03
4,43,裕隆集團_裕隆汽車製造股份有限公司,工程師,2023.02.28,報告很多，實用價值低。會議很多，有開等於沒開，開完一堆追蹤。對主要專精車輛事業不如中華汽車。...,報告很多實用價值低,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2023-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,講真的每天保養維修車輛數又不多輪胎電瓶定位有報價客人也不一定要換因為外場便宜很多,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-12
363,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,況且車子已經賣不好了每個月廠業績越定越高根本很難達到,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-12
364,5930,匯豐汽車股份有限公司,技師,2022.12.12,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？。講真的每天...,建議去別廠牌車子賣得比較好的保養廠做不然匯豐商用客戶已經慢慢流向豐田中華產品也沒在創新在沒幾...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2022-12
365,5931,匯豐汽車股份有限公司,助理,2022.11.11,行政作業居多，內業為主，但偶而要有外出的可能性，光是入貴公司，我的職位因為公司業務考量，直接...,行政作業居多內業為主但偶而要有外出的可能性光是入貴公司我的職位因為公司業務考量直接變更職務依...,0,0,0,1,...,1,0,0,0,0,0,0,1,2,2022-11


In [14]:
dtm_df.to_csv('./salary/salary_sentence_dictword_dtm.csv', encoding='utf-8')

### 計算correlation

In [33]:
df_correlation = pd.DataFrame(np.corrcoef(dtm_co.T),columns=dtm_co.columns, index=dtm_co.columns)

/Users/changchenyu/opt/anaconda3/envs/sma/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/changchenyu/opt/anaconda3/envs/sma/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [34]:
df_corre_vect = df_correlation.reset_index()
df_corre_vect.rename(columns={'index': 'item1'}, inplace=True)
df_corre_vect

,item1,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,成就感,1.000000,-0.000119,-0.014418,0.097580,-0.066284,-0.026427,0.071807,-0.020816,-4.124138e-02,...,0.114720,0.029214,-0.020394,-0.013338,-0.001949,0.040673,-0.019865,-0.050771,0.149264,0.023768
1,學習成長,-0.000119,1.000000,0.001568,-0.027806,-0.007877,0.035791,-0.026147,0.105572,1.569350e-01,...,0.166179,-0.016229,-0.000077,0.179581,0.022597,0.015464,0.118872,0.223697,0.204539,0.234187
2,創新,-0.014418,0.001568,1.000000,-0.000529,-0.023529,-0.005708,-0.037661,0.030787,1.765723e-02,...,0.008538,0.019899,-0.009355,-0.018636,-0.016352,0.069357,0.023298,0.071441,0.075824,0.088018
3,薪資,0.097580,-0.027806,-0.000529,1.000000,0.547227,0.221022,0.084757,-0.062142,-8.415114e-03,...,0.161324,0.176662,-0.047953,0.010263,0.143088,0.036036,-0.040794,-0.004118,0.145981,0.175711
4,福利,-0.066284,-0.007877,-0.023529,0.547227,1.000000,0.163100,0.062302,-0.079826,-9.224759e-02,...,0.138132,0.093797,-0.006533,-0.003836,0.021168,-0.006554,-0.050515,-0.065098,0.177496,0.123148
5,管理制度,-0.026427,0.035791,-0.005708,0.221022,0.163100,1.000000,0.122984,0.034417,4.472717e-02,...,0.195625,0.022969,-0.000133,-0.071439,0.069160,-0.000276,-0.008886,0.059818,0.061815,0.197388
6,工作氛圍,0.071807,-0.026147,-0.037661,0.084757,0.062302,0.122984,1.000000,0.196507,2.556241e-03,...,0.240824,-0.000823,-0.007180,-0.013095,0.115756,0.011742,0.146173,0.056332,0.207227,0.273460
7,同事互動,-0.020816,0.105572,0.030787,-0.062142,-0.079826,0.034417,0.196507,1.000000,3.480725e-02,...,0.170555,-0.025811,0.125959,-0.020259,0.067501,0.081319,0.483063,0.081401,0.206496,0.321326
8,主管風格,-0.041241,0.156935,0.017657,-0.008415,-0.092248,0.044727,0.002556,0.034807,1.000000e+00,...,0.101650,0.009175,0.032112,-0.011783,0.141884,-0.011186,0.111491,0.793046,0.202123,0.347078
9,工作地點,0.005476,-0.018499,0.135169,0.108650,0.077922,0.027548,0.029830,-0.020909,7.971102e-02,...,0.100659,0.127045,0.003553,-0.038128,0.057386,-0.016963,0.047220,0.099956,0.059501,0.177595


In [35]:
corr_melt_df = pd.melt(df_corre_vect, id_vars='item1', value_vars=list(df_corre_vect.columns)[1:], var_name='item2', value_name='correlation')
corr_melt_df.rename(columns={'index':'item1'},inplace=True)
corr_melt_df

,item1,item2,correlation
0,成就感,成就感,1.000000
1,學習成長,成就感,-0.000119
2,創新,成就感,-0.014418
3,薪資,成就感,0.097580
4,福利,成就感,-0.066284
...,...,...,...
3244,出路,負向,0.085161
3245,能力,負向,0.380619
3246,招募人員,負向,0.556911
3247,正向,負向,0.509032


In [36]:
new_dict

{'人際價值': ['工作氛圍', '同事互動', '主管風格'],
 '公司文化與價值': ['多元性', '企業社會責任', '企業永續目標'],
 '情緒': ['正向', '負向'],
 '成長價值': ['成就感', '學習成長', '創新'],
 '招募人員': ['招募人員'],
 '招募資訊': ['面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力'],
 '產品與服務': ['引擎',
  '馬力',
  '避震',
  '外觀',
  '操控',
  '安全',
  '堅固',
  '配備',
  '價錢',
  '科技',
  '品質',
  '折舊',
  '品牌',
  '空間',
  '保養',
  '續航',
  '尺寸',
  '車種',
  '驅動',
  '變速箱'],
 '科系專業': ['電資', '機械', '製造', '車輛工程', '品管'],
 '組織安全與安定': ['工作地點', '公司規模', '工作環境', '產業前景'],
 '經濟報酬': ['薪資', '福利', '管理制度'],
 '變化性': ['輪調', '外派', '出差']}

In [37]:
def get_class_by_subclass(sub_class):
    for k, v in new_dict.items():
        if (sub_class in v):
            return k

In [38]:
corr_melt_df2 = corr_melt_df[~corr_melt_df['correlation'].isnull()].reset_index(drop='index')
corr_melt_df2['class'] = corr_melt_df2['item1'].apply(lambda x: get_class_by_subclass(x))
corr_melt_df2

,item1,item2,correlation,class
0,成就感,成就感,1.000000,成長價值
1,學習成長,成就感,-0.000119,成長價值
2,創新,成就感,-0.014418,成長價值
3,薪資,成就感,0.097580,經濟報酬
4,福利,成就感,-0.066284,經濟報酬
...,...,...,...,...
2596,出路,負向,0.085161,招募資訊
2597,能力,負向,0.380619,招募資訊
2598,招募人員,負向,0.556911,招募人員
2599,正向,負向,0.509032,情緒


In [39]:
corr_melt_df3 = corr_melt_df2[(corr_melt_df2['correlation'] <0) & (corr_melt_df2['correlation'] < 100)]
corr_melt_df3

,item1,item2,correlation,class
1,學習成長,成就感,-0.000119,成長價值
2,創新,成就感,-0.014418,成長價值
4,福利,成就感,-0.066284,經濟報酬
5,管理制度,成就感,-0.026427,經濟報酬
7,同事互動,成就感,-0.020816,人際價值
...,...,...,...,...
2512,輪調,正向,-0.038396,變化性
2529,變速箱,正向,-0.008944,產品與服務
2566,引擎,負向,-0.021941,產品與服務
2571,價錢,負向,-0.019298,產品與服務


In [40]:
a = list(corr_melt_df2['class'].unique())
a

['成長價值',
 '經濟報酬',
 '人際價值',
 '組織安全與安定',
 '變化性',
 '產品與服務',
 '科系專業',
 '公司文化與價值',
 '招募資訊',
 '招募人員',
 '情緒']

In [41]:
a.sort()
a

['人際價值',
 '公司文化與價值',
 '情緒',
 '成長價值',
 '招募人員',
 '招募資訊',
 '產品與服務',
 '科系專業',
 '組織安全與安定',
 '經濟報酬',
 '變化性']

In [42]:
corr_melt_df2.to_csv('interview_corr.csv', encoding='utf-8')

In [43]:
corr_melt_df[corr_melt_df['item1'] == '負向'].sort_values(['correlation'], ascending=False)[1:7]

,item1,item2,correlation
1025,負向,馬力,0.999932
2564,負向,面試,0.580028
3134,負向,招募人員,0.556911
3191,負向,正向,0.509032
3077,負向,能力,0.380619
2735,負向,工作,0.360637


### 計算 co-occurrence

In [178]:
result = np.dot(dtm_co.T.values, dtm_co.values)
co_occurrence_df = pd.DataFrame(result, columns=dtm_co.columns, index=dtm_co.columns)
co_occurrence_df.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
成就感,19,9,0,14,1,9,3,4,19,0,...,38,3,0,1,1,0,12,32,77,4169
學習成長,9,338,4,54,51,182,56,105,448,24,...,397,17,2,40,9,22,336,741,790,72241
創新,0,4,6,0,0,6,0,6,1,0,...,6,0,0,0,0,0,16,8,30,1596
薪資,14,54,0,592,243,303,74,32,318,49,...,414,43,0,16,23,16,206,498,716,66516
福利,1,51,0,243,248,213,52,22,187,25,...,259,16,0,9,4,11,136,311,543,42820


In [181]:
df_ocurr_vect = co_occurrence_df.reset_index()
df_ocurr_vect.rename(columns={'index': 'item1'}, inplace=True)
df_ocurr_vect
ocurr_melt_df = pd.melt(df_ocurr_vect, id_vars='item1', value_vars=list(df_ocurr_vect.columns)[1:], var_name='item2', value_name='ocurrelation')
ocurr_melt_df.rename(columns={'index':'item1'},inplace=True)
ocurr_melt_df

,item1,item2,ocurrelation
0,成就感,成就感,19
1,學習成長,成就感,9
2,創新,成就感,0
3,薪資,成就感,14
4,福利,成就感,1
...,...,...,...
3244,出路,負向,10597
3245,能力,負向,221842
3246,招募人員,負向,482379
3247,正向,負向,525814
